In [1]:
import pandas as pd
import numpy as np
from dask import dataframe as dd

In [2]:
#File location S3
#loc = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2010-base-20200206/beam/sfbay-pilates-base__2022-02-06_07-26-58_kwy/ITERS/it.0/"
locSF_rh = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2010-lessrh-20220223/beam/year-2010-iteration-2/ITERS/it.0/"
loc_2018 =  "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220306/beam/year-2018-iteration-5/ITERS/it.0/"

In [3]:
%%time
#Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "reason": "category",
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(locSF_rh + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:21: DtypeWarning: Columns (1,2,3,21,23,24,30,31,37,52) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 2min 34s


In [4]:
#Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
#Rename mode column
eventsSF.rename(columns={"mode":"modeType"}, inplace=True)  

In [6]:
#Replace "Work" with "work" in actType column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [8]:
Walk_transit = eventsSF[(eventsSF['person']=="4199718")|(eventsSF['driver']=="4199718")|(eventsSF['vehicle'] == "AC:5571737")|(eventsSF['vehicle'] == "AC:5595297")|(eventsSF['vehicle'] == "AC:5572611")
         |(eventsSF['vehicle'] =='AC:5572612')|(eventsSF['vehicle'] =='AC:5595644')|(eventsSF['vehicle'] =='AC:5572381')
        |(eventsSF['vehicle'] =='AC:5572465')|(eventsSF['vehicle'] =='BA:70SFO11')|(eventsSF['person']=="TransitDriverAgent-AC:5572612")]

In [9]:
#Walk_transit.to_csv('C:/Shared-Work/Data/CleanData/Walk_transit_3_9.csv', index = False)

In [10]:
#Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan) 

In [11]:
#Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [12]:
#Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [13]:
eventsSFCopy = eventsSF.copy()

In [14]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(6234, 62)

In [ ]:
eventsSF[eventsSF['vehicle'] == '236306']

In [16]:
longwalk2= eventsSF[eventsSF['IDMerged'] == '181']

In [17]:
longwalk_576_BEAMevents = eventsSF[eventsSF['IDMerged'] == '576']

In [18]:
longwalk_576_BEAMevents.to_csv('C:/Shared-Work/Data/CleanData/longwalk_576_BEAMevents.csv', index = False)

In [ ]:
%%time
#Split the riders column in order to replace transit and ridehail driver with rider id
dfRiders = eventsSF['riders'].str.split(':',expand=True)

In [20]:
riderStartIndex = len(eventsSF.columns)

In [21]:
#Concat the new riders columns to the main dataframe
eventsSF = pd.concat([eventsSF, dfRiders.reindex(eventsSF.index)], axis=1)

NameError: name 'dfRiders' is not defined

In [ ]:
%%time
# Making a riderID column and merge it with the already merged id column 
NaN = np.nan
eventsSF['ID'] = NaN 
    
for col in eventsSF.columns[riderStartIndex:-2]:    #maybe change col to row to make new rows
    eventsSF['riderID'] = np.where(eventsSF[col].isin(eventsSF["IDMerged"]), eventsSF[col], np.nan)
    eventsSF['ID'] = eventsSF['ID'].combine_first(eventsSF['riderID'])

In [ ]:
eventsSF['IDMerged'] = eventsSF['IDMerged'].combine_first(eventsSF['ID'])

In [ ]:
#dropping unused riders columns
eventsSF = eventsSF.drop(columns=eventsSF.columns[riderStartIndex:])

In [ ]:
# shift column 'Name' to first position
first_column = eventsSF.pop('IDMerged')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [ ]:
#SF_chunk = eventsSFnum[0:10000]
#SF_chunk.to_csv('C:/Shared-Work/Data/CleanData/SF_chunk.csv', index = False)

In [ ]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

In [ ]:
#Filling NANs in ID related to charging events
eventsSF["IDMerged"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

In [ ]:
eventsSF[eventsSF['vehicle'] == '236306']

In [ ]:
%%time
#To select just the numeric MergedID
#eventsSF.loc[eventsSF.IDMerged.str.isnumeric()]
eventsSFnum = eventsSF[pd.to_numeric(eventsSF.IDMerged, errors='coerce').notnull()].sort_values(['IDMerged','time']).reset_index(drop=True)

In [ ]:
eventsSFnum['actEndTime'] = np.where(eventsSFnum['type']=='actend'
                     , eventsSFnum['time'], np.nan)

In [ ]:
eventsSFnum['actStartTime'] = np.where(eventsSFnum['type']=='actstart'
                     , eventsSFnum['time'], np.nan)    

In [ ]:
eventsSFnum['PTduration'] = np.where(eventsSFnum['type']=='PathTraversal'
                     , eventsSFnum['arrivalTime'] - eventsSFnum['departureTime'], np.nan)

In [ ]:
eventsSFnum['duration_walking'] = np.where(eventsSFnum['modeType']=='walk'
                     , eventsSFnum['PTduration'], np.nan)

In [ ]:
eventsSFnum['duration_in_privateCar'] = np.where((eventsSFnum['modeType']=='car')|(eventsSFnum['modeType']=='car_hov3')|(eventsSFnum['modeType']=='car_hov2')|
                                              (eventsSFnum['modeType']=='hov2_teleportation')|(eventsSFnum['modeType']=='hov3_teleportation')
                                                 , eventsSFnum['PTduration'], np.nan)

In [ ]:
eventsSFnum['duration_on_bike'] = np.where(eventsSFnum['modeType']=='bike', eventsSFnum['PTduration'], np.nan)

In [ ]:
eventsSFnum['duration_in_ridehail'] = np.where((eventsSFnum['modeType']=='ride_hail')|(eventsSFnum['modeType']=='ride_hail_pooled'), eventsSFnum['PTduration'], np.nan)

In [ ]:
eventsSFnum['duration_in_public'] = np.where((eventsSFnum['modeType']=='bike_transit')|(eventsSFnum['modeType']=='drive_transit')|(eventsSFnum['modeType']=='walk_transit'), eventsSFnum['PTduration'], np.nan)

In [ ]:
print(eventsSFnum.modeType.unique().tolist())

In [ ]:
eventsSFnum["tripIndex"] = eventsSFnum.groupby("IDMerged")["tourIndex"].rank(method="first", ascending=True)
eventsSFnum["tripIndex"] = eventsSFnum.tripIndex.fillna(method='ffill')

In [ ]:
eventsSFnum["currentTourModeFillna"] = eventsSFnum.currentTourMode.fillna(method='ffill')#.astype('object')

In [ ]:
#Change column type to numeric to be able to sort_values
eventsSFnum["IDMerged"] = pd.to_numeric(eventsSFnum["IDMerged"])

In [ ]:
eventsSFnum['actEndType'] = np.where(eventsSFnum['type']=='actend'
                     , eventsSFnum['actType'], "")

In [ ]:
eventsSFnum['actStartType'] = np.where(eventsSFnum['type']=='actstart'
                     , eventsSFnum['actType'], "")

In [ ]:
#SF_chunk = eventsSFnum[0:500]
#SF_chunk.to_csv('C:/Shared-Work/Data/CleanData/SF_chunk.csv', index = False)

In [ ]:
eventsSFtrips = pd.pivot_table(
   eventsSFnum,
   index=['IDMerged','tripIndex','currentTourModeFillna'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'PTduration': np.sum, 'primaryFuel': np.sum, 
            'netCost': np.sum, 'actStartType': np.sum, 'actEndType': np.sum, 'duration_walking': np.sum,
            'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 'duration_in_ridehail': np.sum, 
           'duration_in_public': np.sum}).reset_index()

In [ ]:
eventsSFtrips['DDtime'] = eventsSFtrips['actStartTime'] - eventsSFtrips['actEndTime'] 

In [ ]:
eventsSFtrips['waitTime'] = eventsSFtrips['DDtime'] - eventsSFtrips['PTduration'] 

In [ ]:
######?????
eventsSFtrips['duration_wait'] = eventsSFtrips['DDtime'] - eventsSFtrips['PTduration'] 
######?????

In [ ]:
eventsSFtrips['actPurpose'] = eventsSFtrips['actEndType'] + "_to_" + eventsSFtrips['actStartType']

In [ ]:
eventsSFtrips = eventsSFtrips.rename(columns={'currentTourModeFillna': 'currentTourMode'})

In [ ]:
eventsSFtrips.head(1000)

In [ ]:
temp = eventsSFtrips[eventsSFtrips['duration_walking'] > 10000]

In [ ]:
temp.shape

In [ ]:
eventsSFtrips_chunk = eventsSFtrips[0:1000]
eventsSFtrips_chunk.to_csv('C:/Shared-Work/Data/CleanData/eventsSFtrips_chunk.csv', index = False)

In [ ]:
eventsSFtrips[eventsSFtrips['currentTourMode'] == "bike_transit"]

In [ ]:
s8 = eventsSFnum[eventsSFnum['IDMerged']==8]

In [ ]:
#sfcomplex.to_csv('C:/Shared-Work/Data/CleanData/sfcomplex.csv', index = False)

In [ ]:
sfcomplex = eventsSFnum[eventsSFnum['IDMerged']==1840884]